# miRNA Analysis v2

This notebook performs miRNA analysis with updated methods and additional steps as outlined in the miRNA-Manuscript-Revision.md file.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import roc_curve, auc
import os


In [ ]:
import os
import pandas as pd



import numpy as np


from sklearn.model_selection import train_test_split



from sklearn.preprocessing import RobustScaler


import pickle



# 1. Load the raw data



data_path = (
    "data/raw/Kasim2024-son-veri.csv"  # Correct path based on repository structure
)


df = pd.read_csv(data_path)



# 2. Check for missing data


missing_data = df.isnull().sum()



print(f"Missing data per column:\n{missing_data}")  # Report findings


# Handle missing data (if any). Examples (choose one if needed):


if missing_data.sum() > 0:

    # Mean imputation:

    # df.fillna(df.mean(), inplace=True)

    # Median imputation:

    # df.fillna(df.median(), inplace=True)

    print("Missing values imputed using [chosen method].")  # Explain chosen method


else:

    print("No missing values found.")



# 3. Convert 'SEX' and 'GROUP'


df["SEX"] = df["SEX"].map({"F": 0, "M": 1}).astype("category")  # Female: 0, Male: 1



df["GROUP"] = df["GROUP"].astype("category")


print(f"Data types after conversion:\n{df.dtypes}")



# 4. Robust scaling normalization (on the entire dataset initially)


targets = [

    "mean_mir146a",

    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]


scaler = RobustScaler()


df[targets] = scaler.fit_transform(df[targets])



# 5. Split data into training and testing sets (after normalization)


X = df[
    targets

    + [

        "SEX",
        "AGE",
        "plaque_index",
        "gingival_index",
        "pocket_depth",
        "bleeding_on_probing",
        "number_of_missing_teeth",
    ]
]



y = df["GROUP"]



# Splitting into training and testing sets


X_train, X_test, y_train, y_test = train_test_split(

    X, y, test_size=0.2, random_state=42, stratify=y
)  # Corrected to stratify by your group label


y_train_numeric = y_train.cat.codes


y_test_numeric = y_test.cat.codes



# Scaling the training data and saving scaling parameters for testing set


X_train_scaled = X_train.copy()


X_test_scaled = X_test.copy()



# Re-initializing the scaler to ensure it's not contaminated by test data


scaler = RobustScaler()


X_train_scaled[targets] = scaler.fit_transform(X_train[targets])


X_test_scaled[targets] = scaler.transform(X_test[targets])  # Apply to test data



# Ensure the 'src' directory exists
os.makedirs("src", exist_ok=True)



# Saving the scaling parameters so that if we want to apply the same procedure to unseen data, we can load the scaler and transform the data directly using this pickle file.



with open("src/scaler.pkl", "wb") as f:

    pickle.dump(scaler, f)



# Create directories if they don't exist



os.makedirs("data/processed", exist_ok=True)


os.makedirs("results/main", exist_ok=True)



# Save preprocessed data (both raw and scaled training/test sets)



df.to_csv("data/processed/normalized_mirna_data.csv", index=False)



# Save raw and scaled train and test datasets into their respective directories.



X_train.to_csv("data/processed/X_train_raw.csv", index=False)


X_test.to_csv("data/processed/X_test_raw.csv", index=False)


y_train.to_csv("data/processed/y_train.csv", index=False)


y_test.to_csv("data/processed/y_test.csv", index=False)


X_train_scaled.to_csv("data/processed/X_train_scaled.csv", index=False)


X_test_scaled.to_csv("data/processed/X_test_scaled.csv", index=False)



# Descriptive Statistics


X_train.describe().to_csv("results/main/descriptive_stats_raw_train.csv")


X_test.describe().to_csv("results/main/descriptive_stats_raw_test.csv")


X_train_scaled.describe().to_csv("results/main/descriptive_stats_scaled_train.csv")


X_test_scaled.describe().to_csv("results/main/descriptive_stats_scaled_test.csv")


Missing data per column:
GROUP                      0
SEX                        0
AGE                        0
plaque_index               0
gingival_index             0
pocket_depth               0
bleeding_on_probing        0
number_of_missing_teeth    0
mean_mir146a               0
mean_mir146b               0
mean_mir155                0
mean_mir203                0
mean_mir223                0
mean_mir381p               0
mean_GAPDH                 0
dtype: int64
No missing values found.
Data types after conversion:
GROUP                      category
SEX                        category
AGE                           int64
plaque_index                float64
gingival_index              float64
pocket_depth                float64
bleeding_on_probing         float64
number_of_missing_teeth       int64
mean_mir146a                float64
mean_mir146b                float64
mean_mir155                 float64
mean_mir203                 float64
mean_mir223                 float64
mean_

In [ ]:
import os
import pandas as pd


import scipy.stats as stats


from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt


import seaborn as sns


from statsmodels.stats.multicomp import pairwise_tukeyhsd



# Load the data from the CSV file into a pandas DataFrame


data_path = r"data/raw/Kasim2024-son-veri.csv"


df = pd.read_csv(data_path)



# Check if 'GROUP' column exists
if "GROUP" not in df.columns:
    raise KeyError("'GROUP' column is missing from the DataFrame")

# 1. ANOVA and post hoc tests on GAPDH
groups = df["GROUP"].unique()


gapdh_data = [df["mean_GAPDH"][df["GROUP"] == group] for group in groups]


fvalue, pvalue = stats.f_oneway(*gapdh_data)


print(
    f"ANOVA results for GAPDH across groups: F-value = {fvalue:.4f}, P-value = {pvalue:.4f}"
)



# Perform Levene's test for homogeneity of variances
levene_stat, levene_p = stats.levene(*gapdh_data)


print(f"Levene's test results: Statistic = {levene_stat:.4f}, P-value = {levene_p:.4f}")



# Perform post hoc Tukey HSD test if ANOVA is significant
if pvalue < 0.05:


    tukey_result = pairwise_tukeyhsd(df["mean_GAPDH"], df["GROUP"], alpha=0.05)

    print("Tukey HSD post hoc test results for GAPDH:")

    print(tukey_result)


else:


    print("ANOVA not significant; no post hoc test performed.")



# Calculate Pearson correlations between GAPDH Ct and clinical parameters


clinical_parameters = [
    "plaque_index",
    "gingival_index",
    "pocket_depth",
    "bleeding_on_probing",
    "number_of_missing_teeth",
]


correlations = {}
for parameter in clinical_parameters:
    correlation, p_value = stats.pearsonr(df["mean_GAPDH"], df[parameter])

    correlations[parameter] = {"correlation": correlation, "p_value": p_value}


    print(
        f"Pearson correlation between GAPDH and {parameter}: {correlation:.4f}, P-value = {p_value:.4f}"
    )



# Save results and justification for raw Ct use


gapdh_results = pd.DataFrame({"ANOVA_F": [fvalue], "ANOVA_p": [pvalue]})


gapdh_results.to_csv("results/supplementary/gapdh_analysis_results.csv", index=False)
correlations_df = pd.DataFrame(correlations).T


correlations_df.to_csv("results/supplementary/gapdh_correlations.csv", index=True)



justification_text = f"""




Justification for Raw Ct Analysis:





Due to the observed significant variability of GAPDH expression across the study groups (ANOVA p-value: {pvalue:.3f}, Tukey HSD results showing significant intergroup differences and effect sizes, Supplementary Table S2), and its correlations with disease severity metrics (Supplementary Figure S1), GAPDH was deemed unsuitable as a reference gene for normalization. Using an unstable reference gene would introduce bias into the analysis, potentially confounding the observed differences in miRNA expression. Therefore, subsequent analyses were performed using raw Ct values. Raw Ct analysis provides a more transparent and unbiased approach when a reliable reference gene cannot be identified. This decision aligns with previous research highlighting the potential for reference gene instability in similar contexts, especially inflammatory conditions such as periodontitis (Dheda et al., 2004; Schmittgen and Zakrajsek, 2000; Li et al., 2019; Peng et al., 2012; Ye et al., 2018). While raw Ct analysis relies on the assumption of similar starting RNA amounts across samples, this limitation is mitigated by our quality control measures during sample processing and robust statistical analyses using non-parametric methods. The variability of GAPDH, as evidenced by its significant correlation with bleeding on probing (Supplementary Figure S1 and Supplementary Table S2), further supports the decision to not use it for normalization.





Raw Ct analysis offers several advantages in this context. First, it avoids introducing additional bias or assumptions associated with alternative normalization methods. Second, it focuses on identifying miRNAs exhibiting relatively *large* fold-change differences between groups, where smaller variations introduced by a potentially unstable reference gene are less likely to significantly impact the overall conclusions. Third, the use of robust statistical methods, namely ANOVA followed by post hoc tests with Benjamini-Hochberg FDR correction for multiple comparisons and effect size calculations (Cohen's d), ensures rigorous comparisons of raw Ct values between groups. Despite its limitations, the transparency of raw Ct analysis combined with our rigorous statistical approach and the specific context of our study, makes it a suitable and reliable approach for identifying candidate miRNA biomarkers with altered expression in periodontal disease. If suitable alternative reference genes or normalization methods had been identified, these would have been used instead of the raw Ct approach.
"""



with open("results/supplementary/raw_ct_justification.txt", "w") as f:

    f.write(justification_text)


ANOVA results for GAPDH across groups: F-value = 190.1558, P-value = 0.0000
Levene's test results: Statistic = 4.5712, P-value = 0.0125
Tukey HSD post hoc test results for GAPDH:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     G      P  -4.5173    0.0 -5.1983 -3.8363   True
     G      S    0.587 0.1056 -0.0939   1.268  False
     P      S   5.1044    0.0  4.4234  5.7853   True
----------------------------------------------------
Pearson correlation between GAPDH and plaque_index: -0.6614, P-value = 0.0000
Pearson correlation between GAPDH and gingival_index: -0.6050, P-value = 0.0000
Pearson correlation between GAPDH and pocket_depth: -0.8144, P-value = 0.0000
Pearson correlation between GAPDH and bleeding_on_probing: -0.7548, P-value = 0.0000
Pearson correlation between GAPDH and number_of_missing_teeth: -0.2378, P-value = 0.0132


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.stats import shapiro, levene, kruskal, mannwhitneyu
import os
import scikit_posthocs as sp
import pingouin as pg

# Load preprocessed data (both raw and scaled training sets)
df_raw = pd.read_csv("data/processed/X_train_raw.csv")
df_scaled = pd.read_csv("data/processed/X_train_scaled.csv")

# Check if 'GROUP' column exists in both DataFrames
if "GROUP" not in df_raw.columns or "GROUP" not in df_scaled.columns:
    raise KeyError("'GROUP' column is missing from the DataFrame")

# Define target miRNAs
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]

# 1. Non-parametric tests (Raw Ct Data)
kruskal_results = {}
mannwhitney_results = {}
effect_sizes_nonparametric = {}

for target in targets:
    # Kruskal-Wallis
    groups = df_raw["GROUP"].unique()
    mirna_data = [df_raw[target][df_raw["GROUP"] == group] for group in groups]
    h_statistic, pvalue = stats.kruskal(*mirna_data)
    kruskal_results[target] = {"H-statistic": h_statistic, "p-value": pvalue}

    # Post hoc Dunn's test if Kruskal-Wallis is significant
    if pvalue < 0.05:
        posthoc = sp.posthoc_dunn(
            df_raw, val_col=target, group_col="GROUP", p_adjust="fdr_bh"
        )
        kruskal_results[target]["Post Hoc p-values"] = posthoc

    # Mann-Whitney U and effect sizes
    mannwhitney_results[target] = {}
    effect_sizes_nonparametric[target] = {}

    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            group1 = df_raw[target][df_raw["GROUP"] == groups[i]]
            group2 = df_raw[target][df_raw["GROUP"] == groups[j]]
            stat, p = mannwhitneyu(group1, group2)
            mannwhitney_results[target][f"{groups[i]} vs {groups[j]}"] = (stat, p)

            # Calculate effect size (Cliff's delta) using pingouin
            effect_size = pg.mwu(group1, group2)["CLES"].values[0]
            effect_sizes_nonparametric[target][
                f"{groups[i]} vs {groups[j]}"
            ] = effect_size

# Save non-parametric test results
nonparametric_results_df = pd.DataFrame.from_dict(
    {
        (target, test): kruskal_results[target][test]
        for target in kruskal_results
        for test in kruskal_results[target]
    },
    orient="index",
)

mwu_df = pd.DataFrame.from_dict(
    {
        (miRNA, comparison): mannwhitney_results[miRNA][comparison]
        for miRNA in mannwhitney_results
        for comparison in mannwhitney_results[miRNA]
    },
    orient="index",
    columns=["MWU-statistic", "p-value"],
).reset_index(level=1)

mwu_df = pd.merge(
    mwu_df,
    pd.DataFrame(effect_sizes_nonparametric).stack().reset_index(),
    left_on=["level_1"],
    right_on=["level_1"],
)
mwu_df.columns = ["Comparison", "MWU-statistic", "p-value", "miRNA", "Effect Size"]

mwu_df.to_csv("results/supplementary/mannwhitney_results.csv")
nonparametric_results_df.to_csv(
    "results/supplementary/nonparametric_test_results.csv", index=True
)

# 2. Parametric tests (Scaled Ct Data)
anova_results = {}
tukey_results = {}
effect_sizes_parametric = {}

for target in targets:
    # ANOVA
    groups = df_scaled["GROUP"].unique()
    mirna_data_scaled = [
        df_scaled[target][df_scaled["GROUP"] == group] for group in groups
    ]

    # Normality check using Shapiro-Wilk test
    normality_pvalues = [shapiro(data)[1] for data in mirna_data_scaled]

    # Homogeneity of variance check using Levene's test
    levene_pvalue = levene(*mirna_data_scaled)[1]

    fvalue, pvalue = stats.f_oneway(*mirna_data_scaled)
    anova_results[target] = {"F-statistic": fvalue, "p-value": pvalue}

    # Post hoc Tukey HSD if ANOVA is significant
    if pvalue < 0.05:
        tukey_result = pairwise_tukeyhsd(
            df_scaled[target], df_scaled["GROUP"], alpha=0.05
        )
        # Adjust p-values using Benjamini-Hochberg
        reject, pvals_corrected, _, _ = multipletests(
            tukey_result.pvalues, method="fdr_bh"
        )
        anova_results[target]["Post Hoc p-values"] = pd.DataFrame(
            tukey_result._results_table.data[1:],
            columns=tukey_result._results_table.data[0],
        )
        anova_results[target]["Post Hoc p-values"]["p-adj"] = pvals_corrected

        # Effect sizes (Cohen's d)
        cohen_d = {}
        for i in range(len(groups)):
            for j in range(i + 1, len(groups)):
                group1 = mirna_data_scaled[i]
                group2 = mirna_data_scaled[j]
                d = pg.compute_effsize(group1, group2, eftype="cohen")
                cohen_d[f"{groups[i]} vs {groups[j]}"] = d
        effect_sizes_parametric[target] = cohen_d

parametric_results_df = pd.DataFrame.from_dict(
    {
        (target, test): anova_results[target][test]
        for target in anova_results
        for test in anova_results[target]
    },
    orient="index",
)

parametric_results_df.to_csv("results/main/parametric_test_results.csv")

# Combine into a single dataframe for the manuscript
parametric_nonparametric_combined = pd.DataFrame()

temp_kruskal = pd.DataFrame(kruskal_results).transpose()
temp_kruskal["miRNA"] = temp_kruskal.index

temp_mwu = mwu_df.copy()
temp_anova = parametric_results_df.copy().reset_index().drop("level_1", axis=1)
temp_anova["miRNA"] = temp_anova["level_0"]
temp_anova = temp_anova.set_index("miRNA")

temp_parametric = pd.DataFrame(effect_sizes_parametric).transpose()
temp_parametric["miRNA"] = temp_parametric.index

parametric_nonparametric_combined = (
    temp_kruskal.merge(temp_anova, left_on="miRNA", right_on="miRNA")
    .merge(temp_parametric, left_on="miRNA", right_on="miRNA")
    .drop("level_0", axis=1)
)
parametric_nonparametric_combined.to_csv(
    "results/main/parametric_nonparametric_tests.csv"
)

# 3. Compare and discuss results (this will be done after all analyses are completed)


KeyError: "'GROUP' column is missing from the DataFrame"

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Load preprocessed data (both raw and scaled training sets)
df_raw = pd.read_csv("data/processed/X_train_raw.csv")
df_scaled = pd.read_csv("data/processed/X_train_scaled.csv")

# Define target miRNAs and clinical parameters
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
    "mean_GAPDH",
]
clinical_params = [
    "AGE",
    "plaque_index",
    "gingival_index",
    "pocket_depth",
    "bleeding_on_probing",
    "number_of_missing_teeth",
]

# 1. Correlation analysis (Raw Data)
correlation_matrix_raw = df_raw[targets + clinical_params].corr(method="pearson")

# Create and save correlation heatmap (raw)
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix_raw, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix Heatmap (Raw Ct Values)")
plt.savefig(
    "results/supplementary/correlation_heatmap_raw.png"
)  # Save to supplementary
plt.close()

# 2. Correlation analysis (Scaled Data)
correlation_matrix_scaled = df_scaled[targets + clinical_params].corr(method="pearson")

# Create and save correlation heatmap (scaled)
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix_scaled, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix Heatmap (Robustly Scaled Ct Values)")
plt.savefig("results/main/correlation_heatmap_scaled.png")  # Save to main results
plt.close()

# Create directories if they don't exist
os.makedirs("results/main", exist_ok=True)
os.makedirs("results/supplementary", exist_ok=True)

# Save correlation matrices
correlation_matrix_raw.to_csv(
    "results/supplementary/correlation_matrix_raw.csv", index=True
)
correlation_matrix_scaled.to_csv(
    "results/main/correlation_matrix_scaled.csv", index=True
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import os

# Load preprocessed data (raw and scaled training/test sets)
X_train_raw = pd.read_csv("data/processed/X_train_raw.csv")
X_test_raw = pd.read_csv("data/processed/X_test_raw.csv")
y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")
X_train_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/processed/X_test_scaled.csv")

# Convert y_train and y_test to numeric if necessary
if pd.api.types.is_categorical_dtype(y_train["GROUP"]):
    y_train_numeric = y_train["GROUP"].cat.codes
    y_test_numeric = y_test["GROUP"].cat.codes
else:
    y_train_numeric = y_train["GROUP"]
    y_test_numeric = y_test["GROUP"]

# Define target miRNAs and groups
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]
groups = ["S", "G", "P"]


# ROC analysis function (with precision-recall)
def roc_analysis(X, y, target_name, comparison_name, data_type):
    # Split data ONLY if raw data is being used (since scaled data is already split into training and test sets for comparison)
    if data_type == "raw":
        X_train, X_test, y_train_temp, y_test_temp = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
    else:
        X_train, X_test, y_train_temp, y_test_temp = (
            X,
            y,
            y_train_numeric,
            y_test_numeric,
        )

    model = LogisticRegression(solver="liblinear")  # Add solver for better convergence.
    model.fit(X_train, y_train_temp)
    y_prob = model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test_temp, y_prob)
    roc_auc = auc(fpr, tpr)

    # Youden index for optimal cutoff
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    sensitivity = tpr[optimal_idx]
    specificity = 1 - fpr[optimal_idx]

    try:  # Try calculating accuracy; if it fails, set it to np.nan
        accuracy = sum(y_test_temp == (y_prob > optimal_threshold)) / len(
            y_test_temp
        )  # Using optimal threshold for classification here
    except Exception as e:
        print(
            f"Error calculating accuracy for {target_name} ({comparison_name}, {data_type}): {e}"
        )
        accuracy = np.nan

    precision, recall, _ = precision_recall_curve(y_test_temp, y_prob)
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label=f"{target_name} (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve: {comparison_name} ({data_type})")
    plt.legend(loc="lower right")

    # Create directories if they don't exist
    os.makedirs(
        f'results/{data_type}/{comparison_name.lower().replace(" ", "_")}',
        exist_ok=True,
    )

    plt.savefig(
        f'results/{data_type}/{comparison_name.lower().replace(" ", "_")}/roc_curve_{target_name}.png'
    )
    plt.close()
    return {
        "AUC": roc_auc,
        "Optimal Cutoff": optimal_threshold,
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "Accuracy": accuracy,
        "PR AUC": pr_auc,
    }


# 1. Refined ROC Analysis (Control vs. Periodontitis - Addressing Class Imbalance, if applicable)
# Since we earlier determined no imbalance, this step isn't needed but is kept as is.  If there were imbalance, you would use the commented out part to handle it.
X_raw_cp = X_train_raw[y_train["GROUP"].isin(["S", "P"])]
X_scaled_cp = X_train_scaled[y_train["GROUP"].isin(["S", "P"])]

y_cp = y_train[y_train["GROUP"].isin(["S", "P"])]
# control_count = np.sum(y_cp == 'S')
# periodontitis_count = np.sum(y_cp == 'P')

# if control_count != periodontitis_count:
# Handle class imbalance (e.g., using SMOTE if applicable)
#    sm = SMOTE(random_state=42)
#    X_raw_resampled, y_resampled = sm.fit_resample(X_raw_cp, y_cp)
#    X_scaled_resampled, _ = sm.fit_resample(X_scaled_cp, y_cp)
#
#    y_cp = y_resampled
#    X_raw_cp = pd.DataFrame(X_raw_resampled, columns=targets) # Scaling after resampling
#    X_scaled_cp = pd.DataFrame(X_scaled_resampled, columns=targets)
# else:
#    print("No class imbalance detected for Control vs. Periodontitis.")
#    X_raw_resampled, X_scaled_resampled, y_resampled = X_raw_cp, X_scaled_cp, y_cp

roc_results_raw_cp = {}
roc_results_scaled_cp = {}
print("Starting ROC analysis")
for target in targets:
    roc_results_raw_cp[target] = roc_analysis(
        X_raw_cp[[target]], y_cp, target, "Control vs. Periodontitis", "raw"
    )  # Correctly applied to the dataset
    roc_results_scaled_cp[target] = roc_analysis(
        X_scaled_cp[[target]], y_cp, target, "Control vs. Periodontitis", "scaled"
    )  # Correctly applied to the dataset


# Create and save ROC results DataFrames. This will also be used for reporting in the manuscript.
pd.DataFrame(roc_results_raw_cp).transpose().to_csv(
    "results/supplementary/roc_results_raw_cp.csv", index=True
)
pd.DataFrame(roc_results_scaled_cp).transpose().to_csv(
    "results/main/roc_results_scaled_cp.csv", index=True
)

# 2. ROC with Top miRNAs (All Comparisons) and 3. ROC Analysis (All miRNAs, All Comparisons)
# will be in the next response due to character limitations.


In [ ]:
# ... (Previous code for ROC analysis - Part 1) ...

# 2 & 3. ROC with Top miRNAs (All Comparisons) and All miRNAs (All Comparisons)

# ROC analysis function (same as before, no changes needed)

# Prepare data for all pairwise comparisons (raw and scaled)
roc_results_raw = {}
roc_results_scaled = {}

for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        comparison_name = f"{groups[i]} vs {groups[j]}"

        # Subset data for current comparison
        group1_indices = y_train["GROUP"] == groups[i]
        group2_indices = y_train["GROUP"] == groups[j]

        X_train_raw_subset = X_train_raw[group1_indices | group2_indices][
            targets
        ].values
        X_test_raw_subset = X_test_raw[y_test["GROUP"].isin([groups[i], groups[j]])][
            targets
        ].values
        X_train_scaled_subset = X_train_scaled[group1_indices | group2_indices][
            targets
        ].values
        X_test_scaled_subset = X_test_scaled[
            y_test["GROUP"].isin([groups[i], groups[j]])
        ][targets].values

        y_train_subset = (
            y_train[group1_indices | group2_indices]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )
        y_test_subset = (
            y_test[y_test["GROUP"].isin([groups[i], groups[j]])]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )

        roc_results_raw[comparison_name] = {}
        roc_results_scaled[comparison_name] = {}

        for k, target in enumerate(targets):
            # For correct ROC analysis with train and test datasets
            roc_results_raw[comparison_name][target] = roc_analysis(
                X_train_raw_subset[:, k].reshape(-1, 1),
                X_test_raw_subset[:, k].reshape(-1, 1),
                y_train_subset,
                y_test_subset,
                target,
                comparison_name,
                "raw",
            )
            roc_results_scaled[comparison_name][target] = roc_analysis(
                X_train_scaled_subset[:, k].reshape(-1, 1),
                X_test_scaled_subset[:, k].reshape(-1, 1),
                y_train_subset,
                y_test_subset,
                target,
                comparison_name,
                "scaled",
            )

# Top miRNAs ROC analysis (all comparisons)
top_mirnas = ["mean_mir146b", "mean_mir155", "mean_mir203"]
roc_results_top_mirnas_raw = {}
roc_results_top_mirnas_scaled = {}


for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        comparison_name = f"{groups[i]} vs {groups[j]}"

        # Subset data for current comparison
        group1_indices = y_train["GROUP"] == groups[i]
        group2_indices = y_train["GROUP"] == groups[j]

        X_train_raw_subset = X_train_raw[group1_indices | group2_indices][
            top_mirnas
        ].values
        X_test_raw_subset = X_test_raw[y_test["GROUP"].isin([groups[i], groups[j]])][
            top_mirnas
        ].values
        X_train_scaled_subset = X_train_scaled[group1_indices | group2_indices][
            top_mirnas
        ].values
        X_test_scaled_subset = X_test_scaled[
            y_test["GROUP"].isin([groups[i], groups[j]])
        ][top_mirnas].values

        y_train_subset = (
            y_train[group1_indices | group2_indices]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )
        y_test_subset = (
            y_test[y_test["GROUP"].isin([groups[i], groups[j]])]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )

        roc_results_top_mirnas_raw[comparison_name] = {}
        roc_results_top_mirnas_scaled[comparison_name] = {}

        for k, target in enumerate(top_mirnas):
            roc_results_top_mirnas_raw[comparison_name][target] = roc_analysis(
                X_train_raw_subset[:, k].reshape(-1, 1),
                X_test_raw_subset[:, k].reshape(-1, 1),
                y_train_subset,
                y_test_subset,
                target,
                comparison_name,
                "raw",
            )  # added data_type
            roc_results_scaled[comparison_name][target] = roc_analysis(
                X_train_scaled_subset[:, k].reshape(-1, 1),
                X_test_scaled_subset[:, k].reshape(-1, 1),
                y_train_subset,
                y_test_subset,
                target,
                comparison_name,
                "scaled",
            )  # added data_type

# Save ROC results
# ... (saving of ROC results will be done after combining miRNAs for ROC is calculated, for organization).


In [ ]:
# ... (previous ROC analysis code) ...

# Combining Top miRNAs for ROC Analysis

roc_results_combined_raw = {}
roc_results_combined_scaled = {}
roc_results_combined_raw_then_scaled = {}  # For raw combined, then scaled
roc_results_combined_scaled_then_combined = {}  # For scaled combined, then combined


for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        comparison_name = f"{groups[i]} vs {groups[j]}"

        # Subset data for current comparison
        group1_indices = y_train["GROUP"] == groups[i]
        group2_indices = y_train["GROUP"] == groups[j]

        X_train_raw_subset = X_train_raw[group1_indices | group2_indices][top_mirnas]
        X_test_raw_subset = X_test_raw[y_test["GROUP"].isin([groups[i], groups[j]])][
            top_mirnas
        ]

        X_train_scaled_subset = X_train_scaled[group1_indices | group2_indices][
            top_mirnas
        ]
        X_test_scaled_subset = X_test_scaled[
            y_test["GROUP"].isin([groups[i], groups[j]])
        ][top_mirnas]

        y_train_subset = (
            y_train[group1_indices | group2_indices]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )
        y_test_subset = (
            y_test[y_test["GROUP"].isin([groups[i], groups[j]])]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )

        # Combine raw Ct values, then scale
        X_train_raw_combined = X_train_raw_subset.mean(axis=1).values.reshape(-1, 1)
        X_test_raw_combined = X_test_raw_subset.mean(axis=1).values.reshape(-1, 1)

        X_train_raw_combined_scaled = scaler.fit_transform(X_train_raw_combined)
        X_test_raw_combined_scaled = scaler.transform(X_test_raw_combined)

        # Combine scaled Ct values
        X_train_scaled_combined = X_train_scaled_subset.mean(axis=1).values.reshape(
            -1, 1
        )
        X_test_scaled_combined = X_test_scaled_subset.mean(axis=1).values.reshape(-1, 1)

        roc_results_combined_raw[comparison_name] = roc_analysis(
            X_train_raw_combined,
            X_test_raw_combined,
            y_train_subset,
            y_test_subset,
            "combined_raw",
            comparison_name,
            "raw",
        )
        roc_results_combined_scaled[comparison_name] = roc_analysis(
            X_train_scaled_combined,
            X_test_scaled_combined,
            y_train_subset,
            y_test_subset,
            "combined_scaled",
            comparison_name,
            "scaled",
        )
        roc_results_combined_raw_then_scaled[comparison_name] = roc_analysis(
            X_train_raw_combined_scaled,
            X_test_raw_combined_scaled,
            y_train_subset,
            y_test_subset,
            "combined_raw_scaled",
            comparison_name,
            "scaled",
        )
        roc_results_combined_scaled_then_combined[comparison_name] = roc_analysis(
            X_train_scaled_combined,
            X_test_scaled_combined,
            y_train_subset,
            y_test_subset,
            "combined_scaled_avg",
            comparison_name,
            "scaled",
        )

# Saving ROC Results (all combined in a dictionary):

roc_results_all = {
    "raw": roc_results_raw,
    "scaled": roc_results_scaled,
    "combined_raw": roc_results_combined_raw,
    "combined_scaled": roc_results_combined_scaled,
    "top_mirnas_raw": roc_results_top_mirnas_raw,
    "top_mirnas_scaled": roc_results_top_mirnas_scaled,
    "combined_raw_then_scaled": roc_results_combined_raw_then_scaled,
    "combined_scaled_then_combined": roc_results_combined_scaled_then_combined,
}


# Iterate over each data type and create tables
for data_type, results in roc_results_all.items():
    for comparison, values in results.items():
        # Create directory for the ROC results if it doesn't exist
        os.makedirs(
            f"results/{'main' if 'scaled' in data_type or 'combined' in data_type else 'supplementary'}/roc_{comparison.lower().replace(' ', '_')}_{data_type}",
            exist_ok=True,
        )
        temp = pd.DataFrame(values).transpose()
        temp["miRNA"] = temp.index
        temp.to_csv(
            f"results/{'main' if 'scaled' in data_type or 'combined' in data_type else 'supplementary'}/roc_{comparison.lower().replace(' ', '_')}_{data_type}.csv",
            index=False,
        )

import json

# Save the entire dictionary to a JSON file
with open("results/main/roc_results_all.json", "w") as fp:
    json.dump(roc_results_all, fp, indent=4)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import os

# Load preprocessed data (scaled training set)
df_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]


# 1. PCA (All Components)
pca = PCA()  # Keep all components
pca_result = pca.fit_transform(df_scaled[targets])
explained_variance_pca = pca.explained_variance_ratio_

# Pairwise 2D scatter plots of PCA components
n_components = pca_result.shape[1]
for i in range(n_components):
    for j in range(i + 1, n_components):
        plt.figure(figsize=(8, 6))
        sns.scatterplot(
            x=pca_result[:, i],
            y=pca_result[:, j],
            hue=df_scaled["GROUP"],
            palette="viridis",
        )
        plt.title(f"PCA Scatter Plot (PC{i+1} vs PC{j+1})")
        plt.xlabel(f"Principal Component {i+1}")
        plt.ylabel(f"Principal Component {j+1}")
        plt.savefig(f"results/supplementary/pca_scatter_pc{i+1}_vs_pc{j+1}.png")
        plt.close()

# 3D scatter plot of the first three components
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(
    pca_result[:, 0],
    pca_result[:, 1],
    pca_result[:, 2],
    c=df_scaled["GROUP"].astype("category").cat.codes,
    cmap="viridis",
)
ax.set_title("3D PCA Scatter Plot (PC1 vs PC2 vs PC3)")
ax.set_xlabel("Principal Component 1")
ax.set_ylabel("Principal Component 2")
ax.set_zlabel("Principal Component 3")
plt.savefig("results/supplementary/pca_scatter_3d.png")
plt.close()

# Save explained variance ratios
explained_variance_df = pd.DataFrame(
    {
        "PC": [f"PC{i+1}" for i in range(len(explained_variance_pca))],
        "Explained Variance Ratio": explained_variance_pca,
    }
)
explained_variance_df.to_csv(
    "results/supplementary/pca_explained_variance.csv", index=False
)

# 2. LDA (All Components)
lda = LDA()  # Retaining all components
lda_result = lda.fit_transform(df_scaled[targets], df_scaled["GROUP"])
explained_variance_lda = lda.explained_variance_ratio_


n_components_lda = lda_result.shape[1]

for i in range(n_components_lda):
    for j in range(i + 1, n_components_lda):
        plt.figure(figsize=(8, 6))
        sns.scatterplot(
            x=lda_result[:, i],
            y=lda_result[:, j],
            hue=df_scaled["GROUP"],
            palette="viridis",
        )
        plt.title(f"LDA Scatter Plot (LD{i+1} vs LD{j+1})")
        plt.xlabel(f"Linear Discriminant {i+1}")
        plt.ylabel(f"Linear Discriminant {j+1}")
        plt.savefig(f"results/supplementary/lda_scatter_ld{i+1}_vs_ld{j+1}.png")
        plt.close()

# Save explained variance ratios
explained_variance_lda_df = pd.DataFrame(
    {
        "LD": [f"LD{i+1}" for i in range(len(explained_variance_lda))],
        "Explained Variance Ratio": explained_variance_lda,
    }
)
explained_variance_lda_df.to_csv(
    "results/supplementary/lda_explained_variance.csv", index=False
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
)
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load preprocessed data (scaled training and test sets)
X_train_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/processed/X_test_scaled.csv")

y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")

# Convert GROUP column to numerical using cat.codes
y_train_numeric = y_train["GROUP"].astype("category").cat.codes
y_test_numeric = y_test["GROUP"].astype("category").cat.codes

# Define target miRNAs (features)
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]
X_train = X_train_scaled[targets]
X_test = X_test_scaled[targets]

# Initialize classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(
        max_iter=1000, solver="liblinear"
    ),  # Increased max_iter and added solver for better convergence
    "LDA": LDA(),
    "SVM": SVC(
        probability=True
    ),  # Add probability parameter so that roc_auc_score can be calculated
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Neural Network": MLPClassifier(
        max_iter=1000, random_state=42, early_stopping=True
    ),  # Increased max_iter and add early stopping to avoid overfitting
}

results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train_numeric)
    y_pred = clf.predict(X_test)
    y_prob = (
        clf.predict_proba(X_test)
        if hasattr(clf, "predict_proba")
        else clf.decision_function(X_test)
    )  # Get predicted probabilities or decision function if probabilities are not available

    results[name] = {
        "accuracy": accuracy_score(y_test_numeric, y_pred),
        "precision": precision_score(y_test_numeric, y_pred, average="weighted"),
        "recall": recall_score(y_test_numeric, y_pred, average="weighted"),
        "f1_score": f1_score(y_test_numeric, y_pred, average="weighted"),
    }
    # Compute roc_auc_score only for binary and multiclass problems
    if len(set(y_test_numeric)) <= 2:
        try:
            results[name]["auc"] = roc_auc_score(y_test_numeric, y_prob)
        except ValueError:
            print(
                "Only one class present in y_true. ROC AUC score is not defined in that case. Skipping AUC calculation."
            )
    elif len(set(y_test_numeric)) > 2:
        results[name]["auc"] = roc_auc_score(y_test_numeric, y_prob, multi_class="ovr")

    # Add classification report to show precision, recall and f1-score for each class
    results[name]["classification_report"] = classification_report(
        y_test_numeric, y_pred, output_dict=True, zero_division=0
    )

    # Add confusion matrix as well
    cm = confusion_matrix(y_test_numeric, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm,
        annot=True,
        cmap="Blues",
        fmt="g",
        xticklabels=sorted(y.unique()),
        yticklabels=sorted(y.unique()),
    )
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: {name}")
    plt.savefig(f"results/main/{name}_cm.png")

# Feature importance for Random Forest
feature_importance = classifiers["Random Forest"].feature_importances_
feature_importance_df = pd.DataFrame(
    {"Feature": targets, "Importance": feature_importance}
).sort_values(by="Importance", ascending=False)

# Create directories if they don't exist
os.makedirs("results/main", exist_ok=True)

# Convert the dictionaries to DataFrames and save the results
pd.DataFrame(results).transpose().to_csv(
    "results/main/classification_results.csv", index=True
)

feature_importance_df.to_csv("results/main/feature_importance.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle


# Load preprocessed data (raw and scaled training/test sets)
X_train_raw = pd.read_csv("data/processed/X_train_raw.csv")
X_test_raw = pd.read_csv("data/processed/X_test_raw.csv")

y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")

X_train_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/processed/X_test_scaled.csv")

# Convert y_train and y_test to numeric if necessary
if pd.api.types.is_categorical_dtype(y_train["GROUP"]):
    y_train_numeric = y_train["GROUP"].cat.codes
    y_test_numeric = y_test["GROUP"].cat.codes
else:
    y_train_numeric = y_train["GROUP"]
    y_test_numeric = y_test["GROUP"]
# Define target miRNAs and groups
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]
top_targets = [
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
]  # Based on prior feature selection
groups = ["S", "G", "P"]


# ROC analysis function (same as before)
# ... (include the roc_analysis function from the previous code block) ...


# 1. Combined miRNA Scores (using training data only for proper assessment)
X_train_combined = X_train_scaled.copy()
X_test_combined = X_test_scaled.copy()

# Simple Average
X_train_combined["combined_avg"] = X_train_scaled[top_targets].mean(axis=1)
X_test_combined["combined_avg"] = X_test_scaled[top_targets].mean(axis=1)

# Weighted Average (using feature importances from Random Forest)
with open("src/scaler.pkl", "rb") as f:
    scaler_loaded = pickle.load(f)
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled[targets], y_train_numeric)  # Using y_train_numeric here
feature_importances = rf.feature_importances_
top_mirna_indices = [targets.index(mirna) for mirna in top_targets]
top_mirna_importances = feature_importances[top_mirna_indices]

X_train_combined["combined_weighted"] = np.dot(
    X_train_scaled[top_targets], top_mirna_importances / np.sum(top_mirna_importances)
)
X_test_combined["combined_weighted"] = np.dot(
    X_test_scaled[top_targets], top_mirna_importances / np.sum(top_mirna_importances)
)

# PCA (First Principal Component)
pca = PCA(n_components=1)
X_train_combined["combined_pca"] = pca.fit_transform(X_train_scaled[top_targets])
X_test_combined["combined_pca"] = pca.transform(X_test_scaled[top_targets])

# LDA (First Linear Discriminant)
lda = LDA(n_components=1)
X_train_combined["combined_lda"] = lda.fit_transform(
    X_train_scaled[top_targets], y_train.values.ravel()
)  # Needs numerical target values for training
X_test_combined["combined_lda"] = lda.transform(X_test_scaled[top_targets])

combined_methods = ["combined_avg", "combined_weighted", "combined_pca", "combined_lda"]

# 2. ROC Analysis with Combined Scores (Raw and Scaled, All Comparisons)
# Initialize dictionaries to store ROC results
roc_results_combined_raw = {}
roc_results_combined_scaled = {}

for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        comparison = f"{groups[i]} vs {groups[j]}"

        # ROC for Averaging Raw Values and Then Scaling as done earlier.
        # This is done before to show that even if averaging is done using raw values, robust normalization still enhances the AUC, however marginally.
        temp_X_raw = X_train_raw[targets].copy()
        temp_X_test = X_test_raw[targets].copy()

        temp_X_raw["combined_top_raw"] = temp_X_raw[top_targets].mean(axis=1)
        temp_X_test["combined_top_raw"] = temp_X_test[top_targets].mean(axis=1)

        # Subset the combined data, scale and perform ROC using the scaled data.
        combined_X_train_raw = temp_X_raw[
            y_train["GROUP"].isin([groups[i], groups[j]])
        ]["combined_top_raw"].values.reshape(-1, 1)
        combined_X_test_raw = temp_X_test[y_test["GROUP"].isin([groups[i], groups[j]])][
            "combined_top_raw"
        ].values.reshape(-1, 1)

        combined_X_train_raw_scaled = scaler_loaded.fit_transform(combined_X_train_raw)
        combined_X_test_raw_scaled = scaler_loaded.transform(combined_X_test_raw)

        y_train_subset = (
            y_train[y_train["GROUP"].isin([groups[i], groups[j]])]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )
        y_test_subset = (
            y_test[y_test["GROUP"].isin([groups[i], groups[j]])]["GROUP"]
            .apply(lambda x: 0 if x == groups[i] else 1)
            .values
        )

        # Initialize the result dictionaries for the current comparison.
        roc_results_combined_raw[comparison] = {}
        roc_results_combined_scaled[comparison] = {}

        roc_results_combined_raw[comparison]["combined_top_raw_scaled"] = roc_analysis(
            combined_X_train_raw_scaled,
            combined_X_test_raw_scaled,
            y_train_subset,
            y_test_subset,
            "combined_top_raw_scaled",
            comparison,
            "scaled",
        )

        for method in combined_methods:
            # Subset data for current comparison
            current_X_train = X_train_combined[
                y_train["GROUP"].isin([groups[i], groups[j]])
            ][method].values.reshape(-1, 1)
            current_X_test = X_test_combined[
                y_test["GROUP"].isin([groups[i], groups[j]])
            ][method].values.reshape(-1, 1)

            roc_results_combined_scaled[comparison][method] = roc_analysis(
                current_X_train,
                current_X_test,
                y_train_subset,
                y_test_subset,
                method,
                comparison,
                "scaled",
            )

# Saving ROC Results (all methods together):
roc_results_combined_all = {
    "combined_raw": roc_results_combined_raw,
    "combined_scaled": roc_results_combined_scaled,
}

# Iterate over each data type and create tables
for data_type, results in roc_results_combined_all.items():
    for comparison, values in results.items():
        # Create directory for the ROC results if it doesn't exist
        os.makedirs(
            f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}",
            exist_ok=True,
        )
        temp = pd.DataFrame(values).transpose()
        temp["method"] = temp.index
        temp.to_csv(
            f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}.csv",
            index=False,
        )

# Save the entire dictionary to a JSON file
import json

with open("results/main/roc_results_combined_all.json", "w") as fp:
    json.dump(roc_results_combined_all, fp, indent=4)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import os

# Load preprocessed data (scaled training and testing sets)
X_train_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/processed/X_test_scaled.csv")

y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")

# Convert y_train and y_test to numeric if necessary
if pd.api.types.is_categorical_dtype(y_train["GROUP"]):
    y_train_numeric = y_train["GROUP"].cat.codes
else:
    y_train_numeric = y_train["GROUP"]

    # Define target miRNAs (features)
    targets = [
        "mean_mir146a",
        "mean_mir146b",
        "mean_mir155",
        "mean_mir203",
        "mean_mir223",
        "mean_mir381p",
    ]
    X_train = X_train_scaled[targets]
    X_test = X_test_scaled[targets]

    # 1. PCA (All Components)
    pca = PCA()  # Keep all components
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    explained_variance_pca = pca.explained_variance_ratio_

    # Pairwise 2D scatter plots of PCA components (on training data)
    n_components = X_train_pca.shape[1]
    for i in range(n_components):
        for j in range(i + 1, n_components):
            plt.figure(figsize=(8, 6))
            sns.scatterplot(
                x=X_train_pca[:, i],
                y=X_train_pca[:, j],
                hue=y_train["GROUP"],
                palette="viridis",
            )
            plt.title(f"PCA Scatter Plot (PC{i+1} vs PC{j+1})")
            plt.xlabel(f"Principal Component {i+1}")
            plt.ylabel(f"Principal Component {j+1}")
            plt.savefig(f"results/supplementary/pca_scatter_pc{i+1}_vs_pc{j+1}.png")
            plt.close()

            # 3D scatter plot of the first three components (on training data)
            fig = plt.figure(figsize=(10, 7))
            ax = fig.add_subplot(111, projection="3d")
            ax.scatter(
                X_train_pca[:, 0],
                X_train_pca[:, 1],
                X_train_pca[:, 2],
                c=y_train["GROUP"].astype("category").cat.codes,
                cmap="viridis",
            )
            ax.set_title("3D PCA Scatter Plot (PC1 vs PC2 vs PC3)")
            ax.set_xlabel("Principal Component 1")
            ax.set_ylabel("Principal Component 2")
            ax.set_zlabel("Principal Component 3")
            plt.savefig("results/supplementary/pca_scatter_3d.png")
            plt.close()

            # Save explained variance ratios
            explained_variance_df = pd.DataFrame(
                {
                    "PC": [f"PC{i+1}" for i in range(len(explained_variance_pca))],
                    "Explained Variance Ratio": explained_variance_pca,
                }
            )
            # Create directories if they don't exist
            os.makedirs("results/supplementary", exist_ok=True)

            explained_variance_df.to_csv(
                "results/supplementary/pca_explained_variance.csv", index=False
            )

            # 2. LDA (All Components)
            lda = LDA()  # Retaining all components
            X_train_lda = lda.fit_transform(
                X_train, y_train_numeric
            )  # Correctly use group labels for LDA
            X_test_lda = lda.transform(X_test)
            explained_variance_lda = lda.explained_variance_ratio_

            # Pairwise 2-D scatter plots for LDA
            n_components_lda = X_train_lda.shape[1]
            for i in range(n_components_lda):
                for j in range(i + 1, n_components_lda):
                    plt.figure(figsize=(8, 6))
                    sns.scatterplot(
                        x=X_train_lda[:, i],
                        y=X_train_lda[:, j],
                        hue=y_train["GROUP"],
                        palette="viridis",
                    )
                    plt.title(f"LDA Scatter Plot (LD{i+1} vs LD{j+1})")
                    plt.xlabel(f"Linear Discriminant {i+1}")
                    plt.ylabel(f"Linear Discriminant {j+1}")
                    plt.savefig(
                        f"results/supplementary/lda_scatter_ld{i+1}_vs_ld{j+1}.png"
                    )
                    plt.close()

                    # Save explained variance ratios
                    explained_variance_lda_df = pd.DataFrame(
                        {
                            "LD": [
                                f"LD{i+1}" for i in range(len(explained_variance_lda))
                            ],
                            "Explained Variance Ratio": explained_variance_lda,
                        }
                    )
                    explained_variance_lda_df.to_csv(
                        "results/supplementary/lda_explained_variance.csv", index=False
                    )


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

# Load preprocessed data (scaled training and test sets)
X_train = pd.read_csv("data/processed/X_train_scaled.csv")
X_test = pd.read_csv("data/processed/X_test_scaled.csv")
y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")

# Convert y_train and y_test to numeric if necessary
if pd.api.types.is_categorical_dtype(y_train['GROUP']):
    y_train_numeric = y_train['GROUP'].cat.codes
    y_test_numeric = y_test['GROUP'].cat.codes
else:
    y_train_numeric = y_train['GROUP']
    y_test_numeric = y_test['GROUP']


    # Define target miRNAs (features)
    targets = ['mean_mir146a', 'mean_mir146b', 'mean_mir155', 'mean_mir203', 'mean_mir223', 'mean_mir381p']

    X_train = X_train[targets]
    X_test = X_test[targets]


    # Initialize classifiers
    classifiers = {
        'Logistic Regression': LogisticRegression(max_iter=1000, solver = 'liblinear'),
        'LDA': LDA(),
        'SVM': SVC(probability=True),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'Neural Network': MLPClassifier(max_iter=1000, random_state=42, early_stopping = True)
    }

    results = {}
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train_numeric)
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test) if hasattr(clf, 'predict_proba') else clf.decision_function(X_test)

        results[name] = {
            'accuracy': accuracy_score(y_test_numeric, y_pred),
            'precision': precision_score(y_test_numeric, y_pred, average='weighted'),
            'recall': recall_score(y_test_numeric, y_pred, average='weighted'),
            'f1_score': f1_score(y_test_numeric, y_pred, average='weighted')
        }
        # Compute roc_auc_score only for binary and multiclass problems
        if len(set(y_test_numeric)) <= 2:
            try:
                results[name]['auc'] = roc_auc_score(y_test_numeric, y_prob)
            except ValueError:
                print("Only one class present in y_true. ROC AUC score is not defined in that case. Skipping AUC calculation.")
            elif len(set(y_test_numeric)) > 2:
                results[name]['auc'] = roc_auc_score(y_test_numeric, y_prob, multi_class = 'ovr')

                # Add classification report and confusion matrix
                results[name]['classification_report'] = classification_report(y_test_numeric, y_pred, output_dict=True, zero_division = 0)
                cm = confusion_matrix(y_test_numeric, y_pred)

                plt.figure(figsize=(8,6))
                sns.heatmap(cm, annot=True, cmap = 'Blues', fmt = 'g', xticklabels = sorted(y_test['GROUP'].unique()), yticklabels=sorted(y_test['GROUP'].unique())) # corrected labels
                plt.xlabel('Predicted')
                plt.ylabel('Actual')
                plt.title(f'Confusion Matrix: {name}')
                plt.savefig(f'results/main/{name}_cm.png')
                plt.close()

                # Feature importance for Random Forest
                feature_importance = classifiers['Random Forest'].feature_importances_
                feature_importance_df = pd.DataFrame({'Feature': targets, 'Importance': feature_importance}).sort_values(by='Importance', ascending=False)

                # Create directories if they don't exist
                os.makedirs('results/main', exist_ok=True)

                # Convert the dictionaries to DataFrames and save the results
                pd.DataFrame(results).transpose().to_csv('results/main/classification_results.csv', index = True)
                feature_importance_df.to_csv('results/main/feature_importance.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json

# Load preprocessed data (raw and scaled training/test sets)
X_train_raw = pd.read_csv("data/processed/X_train_raw.csv")
X_test_raw = pd.read_csv("data/processed/X_test_raw.csv")
y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")

X_train_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/processed/X_test_scaled.csv")

# Convert y_train and y_test to numeric if necessary
if pd.api.types.is_categorical_dtype(y_train["GROUP"]):
    y_train_numeric = y_train["GROUP"].cat.codes
    y_test_numeric = y_test["GROUP"].cat.codes
else:
    y_train_numeric = y_train["GROUP"]
    y_test_numeric = y_test["GROUP"]

    # Define target miRNAs and groups
    targets = [
        "mean_mir146a",
        "mean_mir146b",
        "mean_mir155",
        "mean_mir203",
        "mean_mir223",
        "mean_mir381p",
    ]
    top_targets = ["mean_mir146b", "mean_mir155", "mean_mir203"]
    groups = ["S", "G", "P"]

    # ROC analysis function (same as before)

    def roc_analysis(
        X_train, X_test, y_train, y_test, target_name, comparison_name, data_type
    ):
        model = LogisticRegression(solver="liblinear")
        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        sensitivity = tpr[optimal_idx]
        specificity = 1 - fpr[optimal_idx]
        try:
            accuracy = sum(y_test == (y_prob > optimal_threshold)) / len(y_test)
        except Exception as e:
            print(
                f"Error calculating accuracy for {target_name} ({comparison_name}, {data_type}): {e}"
            )
            accuracy = np.nan

            precision, recall, _ = precision_recall_curve(y_test, y_prob)
            pr_auc = auc(recall, precision)

            return {
                "AUC": roc_auc,
                "Optimal Cutoff": optimal_threshold,
                "Sensitivity": sensitivity,
                "Specificity": specificity,
                "Accuracy": accuracy,
                "PR AUC": pr_auc,
            }

            # 1. Combined miRNA Scores (on training data)
            combined_methods = [
                "combined_avg",
                "combined_weighted",
                "combined_pca",
                "combined_lda",
            ]

            X_train_combined = X_train_scaled.copy()
            X_test_combined = X_test_scaled.copy()

            # Simple Average
            X_train_combined["combined_avg"] = X_train_scaled[top_targets].mean(axis=1)
            X_test_combined["combined_avg"] = X_test_scaled[top_targets].mean(axis=1)

            # Weighted Average (using feature importances from Random Forest)
            with open("src/scaler.pkl", "rb") as f:
                scaler_loaded = pickle.load(f)

                rf = RandomForestClassifier(random_state=42)
                rf.fit(
                    X_train_scaled[targets], y_train_numeric
                )  # Fit on numerical labels
                feature_importances = rf.feature_importances_
                top_mirna_indices = [targets.index(mirna) for mirna in top_targets]
                top_mirna_importances = feature_importances[top_mirna_indices]

                X_train_combined["combined_weighted"] = np.dot(
                    X_train_scaled[top_targets],
                    top_mirna_importances / np.sum(top_mirna_importances),
                )
                X_test_combined["combined_weighted"] = np.dot(
                    X_test_scaled[top_targets],
                    top_mirna_importances / np.sum(top_mirna_importances),
                )

                # PCA (First Principal Component)
                pca = PCA(n_components=1)
                X_train_combined["combined_pca"] = pca.fit_transform(
                    X_train_scaled[top_targets]
                )
                X_test_combined["combined_pca"] = pca.transform(
                    X_test_scaled[top_targets]
                )

                # LDA (First Linear Discriminant)
                lda = LDA(n_components=1)
                X_train_combined["combined_lda"] = lda.fit_transform(
                    X_train_scaled[top_targets], y_train_numeric
                )  # Fit on numeric labels
                X_test_combined["combined_lda"] = lda.transform(
                    X_test_scaled[top_targets]
                )

                # 2. ROC Analysis with Combined Scores (Raw and Scaled, All Comparisons)
                roc_results_combined_raw = {}
                roc_results_combined_scaled = {}

                # Iterate through group comparisons
                for i in range(len(groups)):
                    for j in range(i + 1, len(groups)):
                        comparison = f"{groups[i]} vs {groups[j]}"
                        roc_results_combined_raw[comparison] = {}
                        roc_results_combined_scaled[comparison] = {}
                        # Subset data for the comparison, using y_train and y_test values for subsetting
                        train_subset = y_train["GROUP"].isin([groups[i], groups[j]])
                        test_subset = y_test["GROUP"].isin([groups[i], groups[j]])

                        # Raw and scaled data should also be subsetted using both X_train, X_test and also y_train and y_test indices
                        X_train_raw_subset = X_train_raw[train_subset]
                        X_test_raw_subset = X_test_raw[test_subset]

                        X_train_scaled_subset = X_train_scaled[train_subset]
                        X_test_scaled_subset = X_test_scaled[test_subset]
                        # Convert y_train, and y_test labels to numeric
                        y_train_subset = (
                            y_train[train_subset]["GROUP"]
                            .apply(lambda x: 0 if x == groups[i] else 1)
                            .values
                        )
                        y_test_subset = (
                            y_test[test_subset]["GROUP"]
                            .apply(lambda x: 0 if x == groups[i] else 1)
                            .values
                        )

                        # Combine raw values, scale and calculate ROC
                        combined_X_train_raw = (
                            X_train_raw_subset[top_targets]
                            .mean(axis=1)
                            .values.reshape(-1, 1)
                        )
                        combined_X_test_raw = (
                            X_test_raw_subset[top_targets]
                            .mean(axis=1)
                            .values.reshape(-1, 1)
                        )

                        combined_X_train_raw_scaled = scaler.fit_transform(
                            combined_X_train_raw
                        )
                        combined_X_test_raw_scaled = scaler.transform(
                            combined_X_test_raw
                        )

                        roc_results_combined_raw[comparison][
                            "combined_top_raw_scaled"
                        ] = roc_analysis(
                            combined_X_train_raw_scaled,
                            combined_X_test_raw_scaled,
                            y_train_subset,
                            y_test_subset,
                            "combined_top_raw_scaled",
                            comparison,
                            "scaled",
                        )

                        for method in combined_methods:
                            # Subset data for current comparison, using training set for fitting, test set for transforming as for other methods.
                            X_train_subset = X_train_combined[train_subset][
                                method
                            ].values.reshape(-1, 1)
                            X_test_subset = X_test_combined[test_subset][
                                method
                            ].values.reshape(-1, 1)
                            roc_results_combined_scaled[comparison][method] = (
                                roc_analysis(
                                    X_train_subset,
                                    X_test_subset,
                                    y_train_subset,
                                    y_test_subset,
                                    method,
                                    comparison,
                                    "scaled",
                                )
                            )

                            # Saving ROC Results and Correlations with Clinical Parameters: (Will be in the next response due to character limitations).


In [ ]:
# ... (previous code for combining miRNAs and ROC analysis) ...

# Saving ROC Results (all methods together):
roc_results_combined_all = {
    "combined_raw": roc_results_combined_raw,
    "combined_scaled": roc_results_combined_scaled,
}

# Iterate over each data type and create tables
for data_type, results in roc_results_combined_all.items():
    for comparison, values in results.items():
        # Create directory for the ROC results if it doesn't exist
        os.makedirs(
            f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}",
            exist_ok=True,
        )
        temp = pd.DataFrame(values).transpose()
        temp["method"] = temp.index
        temp.to_csv(
            f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}.csv",
            index=False,
        )

# Save the entire dictionary to a JSON file
import json

with open("results/main/roc_results_combined_all.json", "w") as fp:
    json.dump(roc_results_combined_all, fp, indent=4)

# Correlation Analysis for Combined miRNAs (raw and scaled)

# Raw combined miRNA data (average of raw Ct values of top 3 miRNAs)
X_train_raw["combined_raw"] = X_train_raw[top_targets].mean(axis=1)
X_test_raw["combined_raw"] = X_test_raw[top_targets].mean(axis=1)
# Robustly scaled combined miRNA data
X_train_scaled["combined_scaled_top"] = X_train_scaled[top_targets].mean(axis=1)
X_test_scaled["combined_scaled_top"] = X_test_scaled[top_targets].mean(axis=1)

# Calculate correlations for combined miRNAs (both raw and scaled) and then with all clinical parameters.
correlation_matrix_combined_raw = X_train_raw[["combined_raw"] + clinical_params].corr(
    method="pearson"
)
correlation_matrix_combined_scaled = X_train_scaled[
    ["combined_scaled_top"] + clinical_params
].corr(method="pearson")

# Generate and save heatmaps
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix_combined_raw, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap (Combined Raw Ct Values)")
plt.savefig(
    "results/supplementary/correlation_heatmap_combined_raw.png"
)  # Save to supplementary
plt.close()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix_combined_scaled, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap (Combined Scaled Ct Values)")
plt.savefig(
    "results/main/correlation_heatmap_combined_scaled.png"
)  # Save to main results
plt.close()

# Save correlation matrices
correlation_matrix_combined_raw.to_csv(
    "results/supplementary/correlation_matrix_combined_raw.csv", index=True
)
correlation_matrix_combined_scaled.to_csv(
    "results/main/correlation_matrix_combined_scaled.csv", index=True
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load preprocessed data (raw and scaled training/test sets)
X_train_raw = pd.read_csv("data/processed/X_train_raw.csv")
X_test_raw = pd.read_csv("data/processed/X_test_raw.csv")

y_train = pd.read_csv("data/processed/y_train.csv")
y_test = pd.read_csv("data/processed/y_test.csv")

X_train_scaled = pd.read_csv("data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("data/processed/X_test_scaled.csv")

# Convert y_train and y_test to numeric if necessary
if pd.api.types.is_categorical_dtype(y_train["GROUP"]):
    y_train_numeric = y_train["GROUP"].cat.codes
    y_test_numeric = y_test["GROUP"].cat.codes
else:
    y_train_numeric = y_train["GROUP"]
    y_test_numeric = y_test["GROUP"]

# Define target miRNAs and groups
targets = [
    "mean_mir146a",
    "mean_mir146b",
    "mean_mir155",
    "mean_mir203",
    "mean_mir223",
    "mean_mir381p",
]
groups = ["S", "G", "P"]
clinical_params = [
    "AGE",
    "plaque_index",
    "gingival_index",
    "pocket_depth",
    "bleeding_on_probing",
    "number_of_missing_teeth",
]


# ROC analysis function (same as before)
# ... (include roc_analysis from previous code blocks) ...

# Combine top miRNAs for ROC Analysis, using only the Control vs. the other groups.
top_mirnas = ["mean_mir146b", "mean_mir155", "mean_mir203"]
combined_methods = ["combined_avg", "combined_weighted", "combined_pca", "combined_lda"]

roc_results_combined_raw = {}
roc_results_combined_scaled = {}

# Initialize scaler for re-use
scaler = RobustScaler()

for i in range(1, len(groups)):  # Starts from 1 since 0 is Control
    comparison = f"S vs {groups[i]}"
    roc_results_combined_raw[comparison] = {}
    roc_results_combined_scaled[comparison] = {}
    # Subset data for the comparison, using y_train and y_test values for subsetting
    train_subset = y_train["GROUP"].isin(["S", groups[i]])
    test_subset = y_test["GROUP"].isin(["S", groups[i]])

    X_train_raw_subset = X_train_raw[train_subset][targets]
    X_test_raw_subset = X_test_raw[test_subset][targets]

    X_train_scaled_subset = X_train_scaled[train_subset][targets]
    X_test_scaled_subset = X_test_scaled[test_subset][targets]

    # Convert y_train, and y_test labels to numeric
    y_train_subset = (
        y_train[train_subset]["GROUP"].apply(lambda x: 0 if x == "S" else 1).values
    )
    y_test_subset = (
        y_test[test_subset]["GROUP"].apply(lambda x: 0 if x == "S" else 1).values
    )

    # Combine raw values, scale and calculate ROC
    combined_X_train_raw = (
        X_train_raw_subset[top_targets].mean(axis=1).values.reshape(-1, 1)
    )
    combined_X_test_raw = (
        X_test_raw_subset[top_targets].mean(axis=1).values.reshape(-1, 1)
    )

    combined_X_train_raw_scaled = scaler.fit_transform(combined_X_train_raw)
    combined_X_test_raw_scaled = scaler.transform(combined_X_test_raw)

    roc_results_combined_raw[comparison]["combined_top_raw_scaled"] = roc_analysis(
        combined_X_train_raw_scaled,
        combined_X_test_raw_scaled,
        y_train_subset,
        y_test_subset,
        "combined_top_raw_scaled",
        comparison,
        "scaled",
    )

    # Perform simple averaging of scaled values and store for later ROC analysis.
    X_train_combined = {}
    X_test_combined = {}
    X_train_combined["combined_avg"] = X_train_scaled_subset[top_targets].mean(axis=1)
    X_test_combined["combined_avg"] = X_test_scaled_subset[top_targets].mean(axis=1)

    # Weighted average (training data) using raw and scaled values.
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train_scaled_subset[targets], y_train_subset)  # Fit using training set.
    feature_importances = rf.feature_importances_
    top_mirna_indices = [targets.index(mirna) for mirna in top_targets]
    top_mirna_importances = feature_importances[top_mirna_indices]
    X_train_combined["combined_weighted"] = np.dot(
        X_train_scaled_subset[top_targets],
        top_mirna_importances / np.sum(top_mirna_importances),
    )
    X_test_combined["combined_weighted"] = np.dot(
        X_test_scaled_subset[top_targets],
        top_mirna_importances / np.sum(top_mirna_importances),
    )

    # PCA (First Principal Component) on scaled training data
    pca = PCA(n_components=1)
    X_train_combined["combined_pca"] = pca.fit_transform(
        X_train_scaled_subset[top_targets]
    )
    X_test_combined["combined_pca"] = pca.transform(X_test_scaled_subset[top_targets])

    # LDA (First Linear Discriminant) on scaled training data
    lda = LDA(n_components=1)
    X_train_combined["combined_lda"] = lda.fit_transform(
        X_train_scaled_subset[top_targets], y_train_subset
    )
    X_test_combined["combined_lda"] = lda.transform(X_test_scaled_subset[top_targets])

    # For each combination method, calculate ROC analysis
    roc_results_combined_raw[comparison] = (
        {}
    )  # Initialize empty dictionary for the comparison
    roc_results_combined_scaled[comparison] = {}
    for method in combined_methods:  # Using training and testing sets as before.
        X_train_subset = X_train_combined[method].values.reshape(-1, 1)
        X_test_subset = X_test_combined[method].values.reshape(-1, 1)
        roc_results_combined_scaled[comparison][method] = roc_analysis(
            X_train_subset,
            X_test_subset,
            y_train_subset,
            y_test_subset,
            method,
            comparison,
            "scaled",
        )  # Corrected analysis


# Saving ROC Results (all methods together):
roc_results_combined_all = {
    "combined_raw": roc_results_combined_raw,
    "combined_scaled": roc_results_combined_scaled,
}

# Iterate over each data type and create tables
for data_type, results in roc_results_combined_all.items():
    for comparison, values in results.items():
        # Create directory for the ROC results if it doesn't exist
        os.makedirs(
            f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}",
            exist_ok=True,
        )
        temp = pd.DataFrame(values).transpose()
        temp["method"] = temp.index
        temp.to_csv(
            f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}.csv",
            index=False,
        )

import json

# Save the entire dictionary to a JSON file
with open("results/main/roc_results_combined_all.json", "w") as fp:
    json.dump(roc_results_combined_all, fp, indent=4)

# 4. ROC Analysis (Top miRNAs and Clinical Parameters) will be added in the next response


In [ ]:
# ... (Previous ROC analysis code - Parts 1, 2 & 3) ...

# 4. ROC Analysis (Top miRNAs and Clinical Parameters)

# Select top clinical parameters based on correlation analysis
# (This can be subjective; choose those with the strongest correlations)

top_clinical_params = [
    "pocket_depth",
    "bleeding_on_probing",
]  # Example – choose your top parameters

# Combine top miRNAs and top clinical parameters for ROC analysis (using robust scaled data)
roc_results_combined_clinical = {}

for i in range(
    1, len(groups)
):  # Iterate through G and P groups (compare against Control 'S')
    comparison_name = f"S vs {groups[i]}"

    # Subset data for current comparison (training data only)
    group1_indices = y_train["GROUP"] == "S"  # Control group
    group2_indices = y_train["GROUP"] == groups[i]  # Current group (G or P)

    X_train_scaled_subset = X_train_scaled[
        group1_indices | group2_indices
    ]  # Get scaled train data
    y_train_subset = y_train[group1_indices | group2_indices]["GROUP"].apply(
        lambda x: 0 if x == "S" else 1
    )  # Create binary labels (0 for S, 1 for other)

    roc_results_combined_clinical[comparison_name] = {}
    # ROC Analysis using only Clinical Parameters
    for param in top_clinical_params:
        roc_results_combined_clinical[comparison_name][param] = roc_analysis(
            X_train_scaled_subset[[param]],
            X_test_scaled[test_subset][[param]],
            y_train_subset,
            y_test_subset,
            param,
            comparison_name,
            "scaled",
        )

    # Combine Scaled Top miRNAs for the current comparison
    combined_top_miRNA_train = (
        X_train_scaled_subset[top_mirnas].mean(axis=1).values.reshape(-1, 1)
    )
    combined_top_miRNA_test = (
        X_test_scaled[test_subset][top_mirnas].mean(axis=1).values.reshape(-1, 1)
    )

    for param in top_clinical_params:
        # Combine top miRNAs and each top clinical parameter (Example - averaging method)
        X_train_combined = (
            pd.DataFrame(
                {
                    "combined_mirna": combined_top_miRNA_train.flatten(),
                    "clinical_param": X_train_scaled_subset[param],
                }
            )
            .mean(axis=1)
            .values.reshape(-1, 1)
        )  # Averaging top miRNAs and current clinical parameter, using scaled data.

        X_test_combined = (
            pd.DataFrame(
                {
                    "combined_mirna": combined_top_miRNA_test.flatten(),
                    "clinical_param": X_test_scaled[test_subset][
                        param
                    ],  # Test data should be transformed with the parameters obtained from training.
                }
            )
            .mean(axis=1)
            .values.reshape(-1, 1)
        )

        combined_name = f"combined_top_mirnas_{param}"

        roc_results_combined_clinical[comparison_name][combined_name] = roc_analysis(
            X_train_combined,
            X_test_combined,
            y_train_subset,
            y_test_subset,
            combined_name,
            comparison_name,
            "scaled",
        )

# Save combined clinical ROC results
# Create directory for the ROC results if it doesn't exist
data_type = "combined_clinical"
os.makedirs(f"results/main/roc_{data_type}", exist_ok=True)

for comparison, values in roc_results_combined_clinical.items():
    temp = pd.DataFrame(values).transpose()
    temp["method"] = temp.index
    temp.to_csv(
        f"results/main/roc_{comparison.lower().replace(' ', '_')}_{data_type}.csv",
        index=False,
    )
